In [14]:
import pandas
from pybliometrics.scopus import ScopusSearch

In [15]:
def run_query():
    scopus_search = ScopusSearch('ISSN( 0022-3891)') and ScopusSearch('TITLE-ABS-KEY ( personality )')
    print(scopus_search.get_results_size())
    return scopus_search

In [16]:
bib_fmt = """
@article{{abbr,
  title = {{{title}}},
  author = {{{author}}},
  year = {{{year}}},
  volume = {{{volume}}},
  issn = {{{issn}}},
  abstract = {{{abstract}}},
  journal = {{{journal}}}
}}
"""

In [18]:

def format_item(result):
    if result.author_ids is not None:
        author_ids = result.author_ids.replace(";", " and ")
    else:
        author_ids = ""

    if result.authkeywords is not None:
        keywords = result.authkeywords.replace(" | ", ", ")
    else:
        keywords = ""

    try:
        cover_date = result.coverDate.split("-")[0]
    except:
        cover_date = 1900

    return(bib_fmt.format(
        abbr=result.doi,
        title=result.title,
        author=author_ids,
        year=cover_date,
        volume=result.issueIdentifier,
        issn=result.issn,
        abstract=result.description,
        journal=result.publicationName,
        keywords=keywords
    ))

In [5]:
def write_author_id_mapping(articles, filename):
    journal_wide_author_to_id = {}
    for article in articles:
        # some articles have blank author fields
        if article.author_ids is not None:
            authors_and_ids = dict(zip(article.author_ids.split(';'), 
                                       article.author_names.split(';')))
            journal_wide_author_to_id = {**authors_and_ids, **journal_wide_author_to_id}
        else:  
            # we can print the articles that have no authors here:
#             print(article)
            pass

    df = pandas.DataFrame(journal_wide_author_to_id.items(), columns=["scopus_id", "author"])
    df.to_csv(filename)

In [20]:
def main_routine():
    scopus_search = run_query()
    
    buf = ""
    for result in scopus_search.results:
        buf += format_item(result)
    #print(buf)
    with open("JPSP_p.bib", "wb") as f:
        f.write(buf.encode())
    #write_author_id_mapping(s.results, "JPA.csv")
    #f.write(buf.encode())

In [100]:
def by_year(start_year, end_year):
    scopus_search = run_query()
    
    buf = ""
    for result in scopus_search.results:
        try:
            year = int(result.coverDate.split("-")[0])
        except:
            year = 1900
        if year >= start_year and year <= end_year:
            buf += format_item(result)
    #print(buf)
    with open(f"JPA-{start_year}-{end_year}.bib", "wb") as f:
        f.write(buf.encode())

In [21]:
main_routine()

KeyboardInterrupt: 

In [101]:
by_year(1970,1979)

4297


In [102]:
by_year(1980,1989)

4297


In [103]:
by_year(1990,1999)

4297


In [104]:
by_year(2000,2009)

4297


In [105]:
by_year(2010,2019)

4297


In [60]:
by_year(2020,2019)

3616
